In [1]:
from geneformer import InSilicoPerturber
from geneformer import InSilicoPerturberStats

from geneformer.tokenizer import TOKEN_DICTIONARY_FILE, ORGANISM
from geneformer.in_silico_perturber_stats import GENE_NAME_ID_DICTIONARY_FILE
from geneformer.in_silico_perturber import ISP_device

import sys

print(f"device: {ISP_device}")
print(f"organism: {ORGANISM}")
print(f"gene to ens dict: {GENE_NAME_ID_DICTIONARY_FILE}")
print(f"token directory : {TOKEN_DICTIONARY_FILE}")


2024-12-25 06:42:15.071019: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-25 06:42:16.540857: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


device: cuda:0
organism: mouse
gene to ens dict: /mnt/keita/data/scRNA-datas/mouse_data/mouse-Geneformer/mouse-genecorpus-20M/data1-v2/tokens/MLM-re_token_dictionary_v3_GeneSymbol_to_EnsemblID.pkl
token directory : /mnt/keita/data/scRNA-datas/mouse_data/mouse-Geneformer/mouse-genecorpus-20M/data1-v2/tokens/MLM-re_token_dictionary_v3.pkl


In [2]:
from datasets import load_from_disk
import numpy as np


organism_data = ORGANISM+"_data"
gene_model1 = "mouse-Geneformer" if organism_data=="mouse_data" else "Geneformer"
gene_model2 = "mouse-geneformer" if organism_data=="mouse_data" else "geneformer"
organ_data = "colorectal_cancer"
parent_directory = "/mnt/keita/data/scRNA-datas/"+organism_data+"/"+gene_model1+"/in_silico/input_disease/"+organ_data+"/tokenized_datas/"

dataset_name = parent_directory+organ_data+"_isp_tokenize_dataset_1.dataset/"

# niigata univ
# dataset_name = parent_directory+"endothelial_cell_isp_24WT-24KO_mouse_tokenize_dataset_1.dataset/"

print(dataset_name)

data = load_from_disk(dataset_name)
print(data)
print("total cells: {}".format(len(data["length"])))

disease_types = np.unique(data["disease"])
print(disease_types)
print(disease_types.shape[0])


import re

str = re.findall("EX_", dataset_name)
if (str != []) and (str[0] == "EX_") :
    level = "cross"
else :
    level = "normal"
print("level: ",level)



/mnt/keita/data/scRNA-datas/mouse_data/mouse-Geneformer/in_silico/input_disease/colorectal_cancer/tokenized_datas/colorectal_cancer_isp_tokenize_dataset_1.dataset/
Dataset({
    features: ['input_ids', 'cell_types', 'organ_major', 'disease', 'length'],
    num_rows: 35085
})
total cells: 35085
['AV' 'cont']
2
level:  normal


In [3]:
import glob
import os
import tqdm
from tqdm.notebook import tqdm
from tqdm import tqdm_notebook as tqdm

rmfiles = glob.glob(dataset_name+"/cache*")
#print(rmfiles)
if rmfiles == [] :
    print("not exist cache files")
else :
    for tqdm_i2, rmfile in zip(tqdm(rmfiles, desc='remove files loop'), rmfiles) :
        os.remove(rmfile)
    print("cacheファイルの削除完了!!")

not exist cache files


In [4]:
import glob
import os
import tqdm
from tqdm.notebook import tqdm
from tqdm import tqdm_notebook as tqdm

rmfiles = glob.glob(dataset_name+"/tmp*")
#print(rmfiles)
if rmfiles == [] :
    print("not exist tmp... files")
else :
    for tqdm_i2, rmfile in zip(tqdm(rmfiles, desc='remove files loop'), rmfiles) :
        os.remove(rmfile)
    print("tmp...ファイルの削除完了!!")

not exist tmp... files


In [5]:
# in silico perturbation in deletion mode to determine genes whose 
# deletion in the dilated cardiomyopathy (dcm) state significantly shifts
# the embedding towards non-failing (nf) state


# select_perturb_type: "delete","overexpress","inhibit","activate"

# "delete": delete gene from rank value encoding
# "overexpress": move gene to front of rank value encoding
# "inhibit": move gene to lower quartile of rank value encoding
# "activate": move gene to higher quartile of rank value encoding


#device = "cuda:0"

#organ = "kidney"


select_perturb_type="delete"

start_state = 'cont'
end_state = 'AV'
alt_state = []


"""
isp = InSilicoPerturber(perturb_type=select_perturb_type,
                        perturb_rank_shift=None,
                        genes_to_perturb=["ENSMUSG00000030963"],
                        combos=0,
                        anchor_gene=None,
                        #model_type="CellClassifier",
                        model_type="Pretrained",
                        num_classes=3,
                        emb_mode="cell",
                        cell_emb_style="mean_pool",
                        filter_data=None,#{"cell_type":["Kidney1","Kidney2","Kidney3"]},
                        cell_states_to_model={'state_key': 'disease', 
                                              'start_state': 'normal', 
                                              'goal_state': 'autosomal dominant polycystic kidney disease', 
                                              'alt_states': ['diabetic kidney disease']},
                        max_ncells=2000,
                        emb_layer=0,
                        forward_batch_size=50,
                        nproc=6)
"""

use_model_type = "CellClassifier" #"Pretrained","GeneClassifier","CellClassifier"
genes_to_perturb_list = [] # Apoe, Fth1, Itgax
GtP = "all" if genes_to_perturb_list == [] else str(len(genes_to_perturb_list))
print(GtP)
print("all" if len(genes_to_perturb_list) == 0 else genes_to_perturb_list)
if GtP != "all" :
    Gname = ""
    for ens in genes_to_perturb_list :
        Gname += ens
    print("Gname: {}".format(Gname))
else :
    Gname = "all"

isp = InSilicoPerturber(perturb_type=select_perturb_type,
                        perturb_rank_shift=None,
                        genes_to_perturb="all" if len(genes_to_perturb_list) == 0 else genes_to_perturb_list,
                        combos=0,
                        anchor_gene=None,
                        #model_type="CellClassifier",
                        model_type=use_model_type,
                        num_classes=disease_types.shape[0],
                        emb_mode="cell",
                        cell_emb_style="mean_pool",
                        filter_data=None,#{"cell_type":["Kidney1","Kidney2","Kidney3"]},
                        cell_states_to_model={'state_key': 'disease', 
                                              'start_state':start_state, 
                                              'goal_state':end_state, 
                                              'alt_states': alt_state}, #alt_statesは，ない場合は[]にする 
                        max_ncells=2500,# ここの数値を全細胞数にすると全細胞で実験を行う．今は2000となっていて，全体の2000細胞をランダムに抽出し，それを実験で使用している．
                        emb_layer=0,
                        forward_batch_size=150,
                        nproc=6)



all
all


In [6]:
# outputs intermediate files from in silico perturbation
"""
isp.perturb_data("path/to/model",
                 "path/to/input_data",
                 "path/to/output_directory",
                 "output_prefix")
"""

# outputs intermediate files from in silico perturbation
if use_model_type == "CellClassifier" :
    #fine_tuned_model = "240330_geneformer_CellClassifier_Cop1KO_L2048_B12_LR5e-05_LScosine_WU500_E10_OadamW_F0_ISP-Cop1KO_GPU2"
    fine_tuned_model = "241225_mouse-geneformer_D240817065523-L6E10_CellClassifier_colorectal_cancer_L2048_B12_LR5e-05_LScosine_WU500_E10_OadamW_F0_ISP-colorectal_cancer"
    start_state = start_state.replace(" ","-")
    end_state = end_state.replace(" ","-")
    
    import os
    DIR_NAME = "/mnt/keita/data/scRNA-datas/"+organism_data+"/"+gene_model1+"/in_silico/output_result/InSilicoPerturber/"+organ_data
    if not os.path.exists(DIR_NAME):
        os.mkdir(DIR_NAME)
    
    DIR_NAME = "/mnt/keita/data/scRNA-datas/"+organism_data+"/"+gene_model1+"/in_silico/output_result/InSilicoPerturber/"+organ_data+"/"+use_model_type
    if not os.path.exists(DIR_NAME):
        os.mkdir(DIR_NAME)
    
    DIR_NAME = "/mnt/keita/data/scRNA-datas/"+organism_data+"/"+gene_model1+"/in_silico/output_result/InSilicoPerturber/"+organ_data+"/"+use_model_type+"/"+fine_tuned_model
    if not os.path.exists(DIR_NAME):
        os.mkdir(DIR_NAME)
        
    DIR_NAME = "/mnt/keita/data/scRNA-datas/"+organism_data+"/"+gene_model1+"/in_silico/output_result/InSilicoPerturber/"+organ_data+"/"+use_model_type+"/"+fine_tuned_model+"/SE"+select_perturb_type+"_ST"+start_state+"_EN"+end_state+"_GtP"+GtP+"-G"+Gname
    if not os.path.exists(DIR_NAME):
        os.mkdir(DIR_NAME)
    
    isp.perturb_data("/mnt/keita/data/prog/jupyter/Geneformer/fine_tuned_models/"+gene_model2+"/in_silico_pretraining/{}/{}/".format(level, fine_tuned_model),
                     dataset_name,
                     "/mnt/keita/data/scRNA-datas/"+organism_data+"/"+gene_model1+"/in_silico/output_result/InSilicoPerturber/"+organ_data+"/"+use_model_type+"/"+fine_tuned_model+"/SE"+select_perturb_type+"_ST"+start_state+"_EN"+end_state+"_GtP"+GtP+"-G"+Gname+"/",
                     "output_in-silico_SE{}_OR{}_ST{}_EN{}_G{}".format(select_perturb_type, organ_data, start_state, end_state, Gname))

elif use_model_type == "Pretrained" :
    pretrain_model = "240426_013449_mouse-geneformer_20M_DV3_PTTMLM_L6_emb256_SL2048_E10_B12_LR0.001_LScosine_WU10000_ACTsilu_Oadamw_DS8"
    start_state = start_state.replace(" ","-")
    end_state = end_state.replace(" ","-")
    
    import os
    DIR_NAME = "/mnt/keita/data/scRNA-datas/"+organism_data+"/"+gene_model1+"/in_silico/output_result/InSilicoPerturber/"+organ_data
    if not os.path.exists(DIR_NAME):
        os.mkdir(DIR_NAME)
    
    DIR_NAME = "/mnt/keita/data/scRNA-datas/"+organism_data+"/"+gene_model1+"/in_silico/output_result/InSilicoPerturber/"+organ_data+"/"+use_model_type
    if not os.path.exists(DIR_NAME):
        os.mkdir(DIR_NAME)
    
    DIR_NAME = "/mnt/keita/data/scRNA-datas/"+organism_data+"/"+gene_model1+"/in_silico/output_result/InSilicoPerturber/"+organ_data+"/"+use_model_type+"/"+pretrain_model
    if not os.path.exists(DIR_NAME):
        os.mkdir(DIR_NAME)
        
    DIR_NAME = "/mnt/keita/data/scRNA-datas/"+organism_data+"/"+gene_model1+"/in_silico/output_result/InSilicoPerturber/"+organ_data+"/"+use_model_type+"/"+pretrain_model+"/SE"+select_perturb_type+"_ST"+start_state+"_EN"+end_state+"_GtP"+GtP
    if not os.path.exists(DIR_NAME):
        os.mkdir(DIR_NAME)
    
    isp.perturb_data("/mnt/keita/data/prog/jupyter/Geneformer/models/{}/models/".format(pretrain_model),
                     dataset_name,
                     "/mnt/keita/data/scRNA-datas/"+organism_data+"/"+gene_model1+"/in_silico/output_result/InSilicoPerturber/"+organ_data+"/"+use_model_type+"/"+pretrain_model+"/SE"+select_perturb_type+"_ST"+start_state+"_EN"+end_state+"_GtP"+GtP+"/",
                     "output_in-silico_SE{}_OR{}_ST{}_EN{}_GtP{}".format(select_perturb_type, organ_data, start_state, end_state, GtP))
    
else :
    print("Select use_model_type!")
    pass


"""
pretrain_model = "231106_geneformer_CellClassifier_kidney_L2048_B12_LR5e-05_LScosine_WU500_E10_OadamW_F0_insilico-PRE231009_121636_geneformer_30M_L6_emb256_SL2048_E20_B12_LR0.001_LScosine_WU10000_Oadamw_DS8"


import os
DIR_NAME = "/mnt/keita/data/scRNA-datas/mouse_datas/in_silico/output_InSilicoPerturber/"+pretrain_model
if not os.path.exists(DIR_NAME):
    os.mkdir(DIR_NAME)


isp.perturb_data("/mnt/keita/data/prog/jupyter/Geneformer/fine_tuning/in_silico_pretraining/{}/".format(pretrain_model),#"/mnt/keita/data/prog/jupyter/Geneformer/models/{}/models/".format(pretrain_model),
                 "/mnt/keita/data/scRNA-datas/mouse_datas/in_silico/three_datas/in_silico_three_mouse_tokenize_dataset_1.dataset",
                 "/mnt/keita/data/scRNA-datas/mouse_datas/in_silico/output_InSilicoPerturber/{}/".format(pretrain_model),
                 "output_in-silico_kidney_normal_diabetic-kidney-disease_autosomal-dominant-polycystic-kidney-disease")
"""



Filter (num_proc=6):   0%|          | 0/2500 [00:00<?, ? examples/s]

Filter (num_proc=6):   0%|          | 0/2500 [00:00<?, ? examples/s]

Filter (num_proc=6):   0%|          | 0/35085 [00:00<?, ? examples/s]

  0%|          | 0/2500 [00:00<?, ?it/s]

Map (num_proc=6):   0%|          | 0/942 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map (num_proc=6):   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map (num_proc=6):   0%|          | 0/200 [00:00<?, ? examples/s]

'\npretrain_model = "231106_geneformer_CellClassifier_kidney_L2048_B12_LR5e-05_LScosine_WU500_E10_OadamW_F0_insilico-PRE231009_121636_geneformer_30M_L6_emb256_SL2048_E20_B12_LR0.001_LScosine_WU10000_Oadamw_DS8"\n\n\nimport os\nDIR_NAME = "/mnt/keita/data/scRNA-datas/mouse_datas/in_silico/output_InSilicoPerturber/"+pretrain_model\nif not os.path.exists(DIR_NAME):\n    os.mkdir(DIR_NAME)\n\n\nisp.perturb_data("/mnt/keita/data/prog/jupyter/Geneformer/fine_tuning/in_silico_pretraining/{}/".format(pretrain_model),#"/mnt/keita/data/prog/jupyter/Geneformer/models/{}/models/".format(pretrain_model),\n                 "/mnt/keita/data/scRNA-datas/mouse_datas/in_silico/three_datas/in_silico_three_mouse_tokenize_dataset_1.dataset",\n                 "/mnt/keita/data/scRNA-datas/mouse_datas/in_silico/output_InSilicoPerturber/{}/".format(pretrain_model),\n                 "output_in-silico_kidney_normal_diabetic-kidney-disease_autosomal-dominant-polycystic-kidney-disease")\n'

In [7]:


ispstats = InSilicoPerturberStats(mode="goal_state_shift",
                                  genes_perturbed="all" if len(genes_to_perturb_list) == 0 else genes_to_perturb_list,
                                  combos=0,
                                  anchor_gene=None,
                                  cell_states_to_model={"state_key": "disease",
                                                        "start_state": start_state,
                                                        "goal_state": end_state,
                                                        "alt_states": alt_state},
                                 )


"""
ispstats = InSilicoPerturberStats(mode="goal_state_shift",
                                  genes_perturbed=["ENSMUSG00000030963"],
                                  combos=0,
                                  anchor_gene=None,
                                  cell_states_to_model={"disease":(["diabetic kidney disease"],["normal"],["autosomal dominant polycystic kidney disease"])})
"""


'\nispstats = InSilicoPerturberStats(mode="goal_state_shift",\n                                  genes_perturbed=["ENSMUSG00000030963"],\n                                  combos=0,\n                                  anchor_gene=None,\n                                  cell_states_to_model={"disease":(["diabetic kidney disease"],["normal"],["autosomal dominant polycystic kidney disease"])})\n'

In [8]:
# extracts data from intermediate files and processes stats to output in final .csv
"""
ispstats.get_stats("path/to/input_data",
                   None,
                   "path/to/output_directory",
                   "output_prefix")
"""
# extracts data from intermediate files and processes stats to output in final .csv

#organ = "kidney"

import os
DIR_NAME = "/mnt/keita/data/scRNA-datas/"+organism_data+"/"+gene_model1+"/in_silico/output_result/InSilicoPerturberStats/"+organ_data
if not os.path.exists(DIR_NAME):
    os.mkdir(DIR_NAME)

DIR_NAME = "/mnt/keita/data/scRNA-datas/"+organism_data+"/"+gene_model1+"/in_silico/output_result/InSilicoPerturberStats/"+organ_data+"/"+use_model_type
if not os.path.exists(DIR_NAME):
    os.mkdir(DIR_NAME)

DIR_NAME = "/mnt/keita/data/scRNA-datas/"+organism_data+"/"+gene_model1+"/in_silico/output_result/InSilicoPerturberStats/"+organ_data+"/"+use_model_type+"/"+fine_tuned_model
if not os.path.exists(DIR_NAME):
    os.mkdir(DIR_NAME)

DIR_NAME = "/mnt/keita/data/scRNA-datas/"+organism_data+"/"+gene_model1+"/in_silico/output_result/InSilicoPerturberStats/"+organ_data+"/"+use_model_type+"/"+fine_tuned_model+"/SE"+select_perturb_type+"_ST"+start_state+"_EN"+end_state+"_GtP"+GtP+"-G"+Gname
if not os.path.exists(DIR_NAME):
    os.mkdir(DIR_NAME)


ispstats.get_stats("/mnt/keita/data/scRNA-datas/"+organism_data+"/"+gene_model1+"/in_silico/output_result/InSilicoPerturber/"+organ_data+"/"+use_model_type+"/"+fine_tuned_model+"/SE"+select_perturb_type+"_ST"+start_state+"_EN"+end_state+"_GtP"+GtP+"-G"+Gname+"/",
                   None,
                   "/mnt/keita/data/scRNA-datas/"+organism_data+"/"+gene_model1+"/in_silico/output_result/InSilicoPerturberStats/"+organ_data+"/"+use_model_type+"/"+fine_tuned_model+"/SE"+select_perturb_type+"_ST"+start_state+"_EN"+end_state+"_GtP"+GtP+"-G"+Gname+"/",
                   "output_in-silico_SE{}_OR{}_ST{}_EN{}_G{}".format(select_perturb_type, organ_data, start_state, end_state, Gname))


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/13785 [00:00<?, ?it/s]

  0%|          | 0/13785 [00:00<?, ?it/s]

In [9]:
import glob
import os
import tqdm
from tqdm.notebook import tqdm
from tqdm import tqdm_notebook as tqdm

rmfiles = glob.glob(dataset_name+"/cache*")
#print(rmfiles)
if rmfiles == [] :
    print("not exist cache files")
else :
    for tqdm_i2, rmfile in zip(tqdm(rmfiles, desc='remove files loop'), rmfiles) :
        os.remove(rmfile)
    print("cacheファイルの削除完了!!")

/tmp/ipykernel_3021056/1805568863.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for tqdm_i2, rmfile in zip(tqdm(rmfiles, desc='remove files loop'), rmfiles) :


remove files loop:   0%|          | 0/21 [00:00<?, ?it/s]

cacheファイルの削除完了!!
